In [ ]:
import os
import csv
from tqdm import tqdm
from IPython.display import clear_output
from obspy.clients.iris import Client
from obspy import UTCDateTime
from obspy.clients.fdsn.mass_downloader import CircularDomain, Restrictions, MassDownloader

In [ ]:
client = Client()

datadir = "./"
wdir="./Download"

# Creating the Download Directory
if not os.path.exists(wdir):
    os.mkdir(wdir)
    
# Define your study site
minlon, maxlon = 90, 104
minlat, maxlat = 16, 30
lonc = (minlon + maxlon) / 2
latc = (minlat + maxlat) / 2

# Define stations provider, after using IRIS, try GFZ
provider1 = "IRIS"
provider2 = "GFZ"  

if not os.path.exists(f"{wdir}/{provider1}"):
    os.mkdir(f"{wdir}/{provider1}")
if not os.path.exists(f"{wdir}/{provider2}"):
    os.mkdir(f"{wdir}/{provider2}")

In [ ]:
def get_mseed_storage_1(network, station, location, channel, starttime, endtime):
    """Use custom function to store miniseed waveforms
    is_in_db function needs to be coded yourself
    """
    return os.path.join(f"{wdir}/{provider1}""/miniseed/{}/{}".format(year, event_fname),
        "{}.{}.{}.{}.mseed".format(network, station,location, channel))

def get_mseed_storage_2(network, station, location, channel, starttime, endtime):
    """Use custom function to store miniseed waveforms
    is_in_db function needs to be coded yourself
    """
    return os.path.join(f"{wdir}/{provider2}""/miniseed/{}/{}".format(year, event_fname),
        "{}.{}.{}.{}.mseed".format(network, station,location, channel))

In [ ]:
# Defining the catalog duration
year1 = 2000
year2 = 2001

# Reading the Events Catalog
catalog = f"{datadir}/Events/Catalog_{year1}-{year2}"

with open(catalog, "r") as evtfiles:
    for i, file in enumerate(evtfiles):
        event = file.split()
        origin_time = UTCDateTime(event[0])
        year = event[0][0:4]
        event_fname = "".join(event[0].split("T")[0].split("-")) + "".join("".join("".join(event[0].split("T")[1].split("Z")).split(".")).split(":"))
        # print(origin_time, year, event_fname)

        if os.path.exists(f"{wdir}/{provider1}/miniseed/{year1}/{event_fname}"):
            continue
        
        # Step 1: Data Selection
        # Search for available stations globally
        domain = CircularDomain(
            latitude=latc, longitude=lonc, minradius=25.0, maxradius=95.0
        )
        
        restrictions = Restrictions(
            # Get data from event origin time to one hour after.
            starttime = origin_time,
            endtime = origin_time + 60*60,

            # If True, miniSEED files with gaps and/or overlaps will be rejected (default is True)
            reject_channels_with_gaps = True,

            # The minimum length of the data as a fraction of the requested time frame (0.0~1.0, default is 0.9)
            minimum_length = 0.8,

            # The minimum inter-station distance (default is 1000)
            minimum_interstation_distance_in_m = 10E3,

            # Priority list for the channels.
            channel_priorities = ('BH?','HH?','EH?'),

            # Priority list for the locations.
            location_priorities = ('', '00', '10')
        )
        
        ### For Provider 1 ###
        # Step 2: Storage Options
        # Use custom function to store miniSEED waveforms
        mseed_storage = get_mseed_storage_1
        # Directly use folder name to store StationXML files
        stationxml_storage = f"{wdir}/{provider1}/stations/{year}"

        # Step 3: Start the Download
        # List of FDSN client names or service URLS
        mdl = MassDownloader(providers = [f"{provider1}"])
        tt = mdl.download(domain, restrictions, mseed_storage = mseed_storage, 
            stationxml_storage=stationxml_storage, print_report = False)
        
        clear_output(wait=False)
        
        ### For Provider 2 ###
        # Step 2: Storage Options
        # Use custom function to store miniSEED waveforms
        mseed_storage = get_mseed_storage_2
        # Directly use folder name to store StationXML files
        stationxml_storage = f"{wdir}/{provider2}/stations/{year}"

        # Step 3: Start the Download
        # List of FDSN client names or service URLS
        mdl = MassDownloader(providers = [f"{provider2}"])
        tt = mdl.download(domain, restrictions, mseed_storage = mseed_storage, 
            stationxml_storage=stationxml_storage, print_report = False)
        
        clear_output(wait=False)